In [2]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [7]:
from IPython.display import clear_output

import collections
import pandas as pd
import pprint

def get_position_by_name(markets):
    state = client.state()

    for market in markets:
        market_id = state.market_name_to_id.get(market)    
        if market_id is None:
            return pd.DataFrame()

        # Fetch full trade history if not already
        if not state.markets[market_id].hasFullTradeHistory:
            client.get_full_trade_history(market_id)
            state = client.state()

        trades = state.markets[market_id].trades

        # Prepare data structure for each user in this market
        position_max_mins = collections.defaultdict(lambda: {
            "buy_size": 0.0,
            "sell_size": 0.0,
            "highest_bid": 0.0,            # track highest price buyer paid
            "lowest_ask": float("inf"),    # track lowest price seller received
        })

        # Iterate over all trades
        for t in trades:
            buyer = state.accounts[t.buyer_id].name
            seller = state.accounts[t.seller_id].name

            # Update buy side
            position_max_mins[buyer]["buy_size"] += t.size
            position_max_mins[buyer]["highest_bid"] = max(
                position_max_mins[buyer]["highest_bid"], 
                t.price
            )

            # Update sell side
            position_max_mins[seller]["sell_size"] += t.size
            position_max_mins[seller]["lowest_ask"] = min(
                position_max_mins[seller]["lowest_ask"], 
                t.price
            )

        # Convert the dictionary into a Pandas DataFrame
        results = {}
        for user_name, info in position_max_mins.items():
            # If lowest_ask remained inf, it means this user hasn't sold
            lowest_ask = info["lowest_ask"] if info["lowest_ask"] != float("inf") else None

            results[user_name] = {
                "bought_volume": info["buy_size"],
                "sold_volume":   info["sell_size"],
                "highest_bid":   info["highest_bid"],
                "lowest_ask":    lowest_ask
            }

        # Create a DataFrame from the results
        df = pd.DataFrame.from_dict(results, orient='index')
        df.index.name = 'user'  # name the index column
        df.reset_index(inplace=True)

        # Print the market, then the table
        print(f"Market: {market}")
        print(df)
        print("")

try:
    while True:
        get_position_by_name(["min_sat_test", "max_sat_test", "sum_sat_test"])
        client.wait_portfolio_update()
        clear_output(wait=True)
except KeyboardInterrupt:
    clear_output(wait=True)

# Run once for a specific market
get_position_by_name(["min_sat_test"])


Market: min_sat_test
        user  bought_volume  sold_volume  highest_bid  lowest_ask
0   sat_mark          13.45         0.52         10.0         4.5
1  sat_Grass           0.00        11.01          0.0         4.0
2    sat_bob           0.53         2.45          5.0         3.5

